# Week05 -

In this week we look at using ensembles of models to improve the performance of our models. We will look at the following:

* RandomForest
* AdaBoost
* Gradiant Boosting
* XG Boosting


## Introduction and Overview


In this notebook, we will reuse the Universal Bank dataset.

This time, we are developing a model to predict whether a customer will accept a personal loan offer. The dataset contains 5000 observations and 14 variables. The data is available on one of my GitHub repos.

## Install and import necessary packages

In [ ]:
# You may need to install xgboost (it's not part of the sklearn package)
# !conda install xgboost 

In [1]:
# import packages
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier, RandomForestRegressor
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from xgboost import XGBClassifier


np.random.seed(1)

## Load data 

In [2]:
df = pd.read_csv('UniversalBank.csv')
df.head(5)

,ID,Age,Experience,Income,ZIP Code,Family,CCAvg,Education,Mortgage,Personal Loan,Securities Account,CD Account,Online,CreditCard
0,1,25,1,49,91107,4,1.6,1,0,0,1,0,0,0
1,2,45,19,34,90089,3,1.5,1,0,0,1,0,0,0
2,3,39,15,11,94720,1,1.0,1,0,0,0,0,0,0
3,4,35,9,100,94112,1,2.7,2,0,0,0,0,0,0
4,5,35,8,45,91330,4,1.0,2,0,0,0,0,0,1


In [3]:
df['Personal Loan'].value_counts()

0    4520
1     480
Name: Personal Loan, dtype: int64

## Explore the dataset

In [4]:
# Explore the dataset
# read the first row of the dataset 
print(df.head())
print(df.columns)
print(df.describe())
print(df.info())

   ID  Age  Experience  Income  ZIP Code  Family  CCAvg  Education  Mortgage  \
0   1   25           1      49     91107       4    1.6          1         0   
1   2   45          19      34     90089       3    1.5          1         0   
2   3   39          15      11     94720       1    1.0          1         0   
3   4   35           9     100     94112       1    2.7          2         0   
4   5   35           8      45     91330       4    1.0          2         0   

   Personal Loan  Securities Account  CD Account  Online  CreditCard  
0              0                   1           0       0           0  
1              0                   1           0       0           0  
2              0                   0           0       0           0  
3              0                   0           0       0           0  
4              0                   0           0       0           1  
Index(['ID', 'Age', 'Experience', 'Income', 'ZIP Code', 'Family', 'CCAvg',
       'Education'

## Clean/transform data (where necessary)

In [5]:
# based on findings from data exploration, we need to clean up colum names, as there are some leading whitespace characters
df.columns = [s.strip() for s in df.columns] 
df.columns

Index(['ID', 'Age', 'Experience', 'Income', 'ZIP Code', 'Family', 'CCAvg',
       'Education', 'Mortgage', 'Personal Loan', 'Securities Account',
       'CD Account', 'Online', 'CreditCard'],
      dtype='object')

Drop the columns we are not using as predictors (see previous notebooks -- we are given a subset of input variables to consider)

In [6]:
df = df.drop(columns=['ID', 'ZIP Code'])

In [7]:
# translation education categories into dummy vars
df = df.join(pd.get_dummies(df['Education'], prefix='Edu', drop_first=True))
df.drop('Education', axis=1, inplace = True)

df.head(3)

,Age,Experience,Income,Family,CCAvg,Mortgage,Personal Loan,Securities Account,CD Account,Online,CreditCard,Edu_2,Edu_3
0,25,1,49,4,1.6,0,0,1,0,0,0,0,0
1,45,19,34,3,1.5,0,0,1,0,0,0,0,0
2,39,15,11,1,1.0,0,0,0,0,0,0,0,0


## Split data intro training and validation sets

In [8]:
# construct datasets for analysis
target = 'Personal Loan'
predictors = list(df.columns)
predictors.remove(target)
X = df[predictors]
y = df[target]

In [9]:
# create the training set and the test set 
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, random_state=1)

Save the recall result from this model

## Prediction with RandomForest (using default parameters)

Like all our classifiers, RandomeForestClassifier has a number of parameters that can be adjusted/tuned. In this example below, we simply accept the defaults. You may want to experiment with changing the defaul values and also use GridSearchCV to explore ranges of values.

* n_estimators: The number of trees in the forsest
    - A deeper tree might increase the performance, but also the complexity and chances to overfit.
    - The value must be an integer greater than 0. Default is 100.  
* max_depth: The maximum depth per tree. 
    - Deeper trees might increase the performance, but also the complexity and chances to overfit.
    - The value must be an integer greater than 0. Default is None, which allows the tree to grow without constraint.
* See the SciKit Learn documentation for more details. https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html


In [10]:
rforest = RandomForestClassifier()

In [11]:
_ = rforest.fit(X_train, y_train)

In [12]:
y_pred = rforest.predict(X_test)

In [13]:
print(f"{'Model':^18}{'Score':^18}")
print("************************************")
print(f"{'>> Recall Score:':18}{recall_score(y_test, y_pred)}")
print(f"{'Accuracy Score: ':18}{accuracy_score(y_test, y_pred)}")
print(f"{'Precision Score: ':18}{precision_score(y_test, y_pred)}")
print(f"{'F1 Score: ':18}{f1_score(y_test, y_pred)}")

recall_score(y_test, y_pred

      Model             Score       
************************************
>> Recall Score:  0.8456375838926175
Accuracy Score:   0.9833333333333333
Precision Score:  0.984375
F1 Score:         0.9097472924187726


Save the recall result from this model

In [14]:
rforest_recall = recall_score(y_test, y_pred)

In [20]:
score_measure = "recall"
kfolds = 5
param_grid = {'max_depth': np.arange(1,80),
              'max_features': ['auto', 'sqrt'],
              'min_samples_leaf': np.arange(1,80),
              'min_samples_split': np.arange(2,80),
              'criterion': ['entropy', 'gini']}
ran_forest = RandomForestClassifier()
rand_search = RandomizedSearchCV(estimator = ran_forest, param_distributions=param_grid, cv=kfolds, n_iter=100,
                                 scoring=score_measure, verbose=1, n_jobs=-1,
                                 return_train_score=True)
_ = rand_search.fit(X_train, y_train)
print(f"The best {score_measure} score is {rand_search.best_score_}")
print(f"... with parameters: {rand_search.best_params_}")

Fitting 5 folds for each of 100 candidates, totalling 500 fits
The best recall score is 0.8730438715513342
... with parameters: {'min_samples_split': 4, 'min_samples_leaf': 3, 'max_features': 'sqrt', 'max_depth': 71, 'criterion': 'entropy'}


In [21]:
y_pred = rand_search.predict(X_test)

In [22]:
print("Recall scores...")

print(f"{'Recall Score:':18}{recall_score(y_test, y_pred)}")
random_recall = recall_score(y_test, y_pred)

Recall scores...
Recall Score:     0.8187919463087249


## Step 6: Summarize results    

As usual -- in this section you provide a recap your approach, results, and discussion of findings. 


In [31]:
print("Recall scores...")

print(f"{'Random Forest:                         ' :18}{rforest_recall}")
print(f"{'Random Forest Tree with Random Search: ':18}{random_recall}")

Recall scores...
Random Forest:                         0.8456375838926175
Random Forest Tree with Random Search: 0.8187919463087249


## As per the above recall scores, The performance of the Random Forest model is slightly better than the Random Forest Tree with Random Search model. The Random Forest model has a recall score of 0.8456, which means that it can correctly identify 84.56% of the positive cases. On the other hand, the Random Forest Tree with Random Search model has a recall score of 0.8188, which means it can identify 81.88% of the positive cases correctly.

## We could add more data to the model can improve its performance. Tuning hyperparameters using GridSearch CV might also imporve the model performance.